In [ ]:
# get model
# !git clone https://github.com/multimodallearning/hierarchical-dense-ssl.git

Cloning into 'hierarchical-dense-ssl'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 179 (delta 86), reused 131 (delta 43), pack-reused 0
Receiving objects: 100% (179/179), 57.28 KiB | 4.09 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
# move model to drive for future use
# !mv "/content/hierarchical-dense-ssl" "/content/drive/MyDrive/"

In [ ]:
# get weights
# !wget https://drive.google.com/drive/folders/106mM3_s7m_wu4EoRr-AhilIJ5-r80thU?usp=drive_link

In [ ]:
# get dataset
# !wget https://zenodo.org/records/7155725/files/amos22.zip?download=1 -O amos22.zip

--2024-03-27 13:06:15--  https://zenodo.org/records/7155725/files/amos22.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24234336519 (23G) [application/octet-stream]
Saving to: ‘amos22.zip’

amos22.zip          100%[===================>]  22.57G  24.6MB/s    in 15m 34s 

2024-03-27 13:21:50 (24.7 MB/s) - ‘amos22.zip’ saved [24234336519/24234336519]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
# # __MACOSX isn't needed. Unzip only amos file
# !unzip amos22.zip "amos22/*"

# # rename
# !mv amos22 amos22_extract

'unzip' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# get pipreq for extracting requirements
# !pip install pipreqs

In [ ]:
# get and download requirements
# !pipreqs hierarchical-dense-ssl --force

!pip install -r /hierarchical-dense-ssl/requirements.txt

# other packages that isn't in requirements.txt
# !pip install reorient-nii
# !pip install clearml
# !pip install pytorch_lightning
# !pip install amid
# !pip install -U tensorboardX
# !pip install -U tensorboard

In [ ]:
# dataset.json need to be put in the same dir as amos.zip file, or amid fails to read raw data
# !cp /content/amos22_extract/dataset.json /content/

In [19]:
# preprocess data for downstream
# !python hierarchical-dense-ssl/preprocess_downstream_data.py

Traceback (most recent call last):
  File "d:\kpdl\code\hierarchical-dense-ssl\preprocess_downstream_data.py", line 72, in <module>
    nib.save(nib.Nifti1Image(data.image(data_id), affine=data.affine(data_id)), output_path_image)
  File "d:\PRTS\llm\cuda\lib\site-packages\connectome\engine\graph.py", line 30, in __call__
    return evaluate(self.output, hashes, cache)
  File "d:\PRTS\llm\cuda\lib\site-packages\connectome\engine\graph.py", line 63, in evaluate
    return execute(Command.Evaluate, node, hashes, cache)
  File "d:\PRTS\llm\cuda\lib\site-packages\connectome\engine\vm.py", line 96, in execute
    push(func(*pos, **kw))
  File "d:\PRTS\llm\cuda\lib\site-packages\amid\amos\dataset.py", line 79, in image
    with unpack(_base / archive_name, file, archive_root, '.zip') as (unpacked, is_unpacked):
  File "C:\Program Files\Python310\lib\contextlib.py", line 135, in __enter__
    return next(self.gen)
  File "d:\PRTS\llm\cuda\lib\site-packages\amid\utils.py", line 47, in unpack
 

# changes need in preprocess_downstream_data.py

```
AMOS_DIR = '/content/'
```
```
OUTPUT_DIR = '/content/output/'
```
```
dataset = 'amos'
spacing_x = 1.5  # 1.0 for ct 1.5 for mri
spacing_y = 1.5  # 1.0 for ct 1.5 for mri
spacing_z = 1.5  # 2.0 for ct 1.5 for mri
```






In [ ]:
# !mv /content/output/ /content/drive/MyDrive/

In [ ]:
# !cp -R /content/drive/MyDrive/output/amos/labels /content/drive/MyDrive/output

In [ ]:
# !rm -rf /content/drive/MyDrive/log_dir/eval

In [8]:
# run code
!python run_doownstream_task.py --dataset amos --ckpt /weights/hb_ssl_mri.ckpt --log_dir ./log_dir --batch_size 7 --setup from_scratch

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type          | Params
-------------------------------------------
0 | backbone | FPN3d         | 47.6 M
1 | head     | FPNLinearHead | 11.9 K
-------------------------------------------
47.6 M    Trainable params
0         Non-trainable params
47.6 M    Total params
190.319   Total estimated model params size (MB)
Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

# changes need in order to run run_doownstream_task.py

*   in downstream_dataset.py
```
AMOS_DATA_DIR = '/content/output/amos/'
```

# TODO

*   check data split's implementation



